# MasterMonitor Sensor Script

## AdaFruit IO Setup

In [1]:
from Adafruit_IO import Client, Feed, RequestError
from dotenv import load_dotenv
import os

load_dotenv()

AdaFruitIO_Username = os.environ.get("ADAFRUIT_IO_USERNAME")
AdaFruitIO_Key = os.environ.get("ADAFRUIT_IO_KEY")

aio = Client(AdaFruitIO_Username, AdaFruitIO_Key)

## Metadata (location coordinates)

In [2]:
metadata = {'lat': 36.017433,
           'lon': -84.239815,
           'ele': 0,
           'created_at': None}

## External Temperature Sensor

### Imports

In [3]:
import smbus

### Setup

In [4]:
def readTemp():
    address = 0x48
    bus = smbus.SMBus(1)
    raw = bus.read_word_data(address, 0) & 0xFFFF
    raw = ((raw << 8) & 0xFF00) + (raw >> 8)
    temperature = (raw / 32.0) / 8.0
    return temperature

## CCS811 CO2 and VOCs Sensor

### Imports

In [5]:
import busio
import board
import adafruit_ccs811

### Setup

In [6]:
i2c = board.I2C()
ccs =  adafruit_ccs811.CCS811(i2c)

while (not ccs.data_ready):
    pass

print ("CCS811 Sensor ready.")

CCS811 Sensor ready.


## Main

### Imports

In [7]:
from bme280 import BME280
import ST7735
from enviroplus.noise import Noise
from ltr559 import LTR559

import time
from IPython.display import clear_output
from board import *

### Variables

In [8]:
soundFeed = aio.feeds("sound")
luxFeed = aio.feeds("light")
intTempCfeed = aio.feeds("inttempc")
intTempFfeed = aio.feeds("inttempf")
extemp2Feed = aio.feeds("extemp2")
extempFeed = aio.feeds("extemp")
humidFeed = aio.feeds("humidity")
pressFeed = aio.feeds("pressure")
tvocsFeed = aio.feeds("new-sensor.tvocs")
co2Feed = aio.feeds("new-sensor.co2")

range=[(100, 1200)]

noise = Noise()
ltr559 = LTR559()
bme280 = BME280()

Amps = noise.get_amplitudes_at_frequency_ranges(range)
Light = ltr559.get_lux()
External = readTemp()
External2 = External * 1.8 + 32
Celcius = bme280.get_temperature()
Fahrenheit = Celcius * 1.8 + 32
InternalC = Celcius - External
InternalF = Fahrenheit - External2
Humidity = bme280.get_humidity()
Pressure = bme280.get_pressure()
CO2 = ccs.eco2
TVOC = ccs.tvoc

time_calculation = time.time()
aio.send_data(co2Feed.key, CO2, metadata)
aio.send_data(tvocsFeed.key, TVOC, metadata)
aio.send_data(soundFeed.key, Amps[0], metadata)
aio.send_data(luxFeed.key, Light, metadata)
aio.send_data(intTempCfeed.key, InternalC, metadata)
aio.send_data(intTempFfeed.key, InternalF, metadata)
aio.send_data(extemp2Feed.key, External2, metadata)
aio.send_data(extempFeed.key, External, metadata)
aio.send_data(humidFeed.key, Humidity, metadata)
aio.send_data(pressFeed.key, Pressure, metadata)
time_elapsed = time.time() - time_calculation

OSError: [Errno 121] Remote I/O error

### Loop

In [ ]:
while True:
    try:
        Amps = noise.get_amplitudes_at_frequency_ranges(range)
        Light = ltr559.get_lux()
        External = readTemp()
        External2 = External * 1.8 + 32
        InternalC = Celcius - External
        InternalF = Fahrenheit - External2
        Humidity = bme280.get_humidity()
        Pressure = bme280.get_pressure()
        CO2 = ccs.eco2
        TVOCs = ccs.tvoc
        output = "Temperature: External: {0:.2f} C, External: {1:.2f} F, Internal: {2:.2f} C, Internal: {3:.2f} F, - Humidity: {4:.2f} % - Pressure: {5:.2f} hPa, - CO2: {6:.2f} ppm, - TVOCs: {7:.2f} ppb".format(External, External2, InternalC, InternalF, Humidity, Pressure, CO2, TVOCs)
        print(output)
        time.sleep(5)
        clear_output()
        time_elapsed = time.time() - time_calculation
        
        if time_elapsed == 5:
            time_calculation = time.time()
            aio.send_data(co2Feed.key, CO2, metadata)
            aio.send_data(tvocsFeed.key, TVOC, metadata)
            aio.send_data(soundFeed.key, Amps[0], metadata)
            aio.send_data(luxFeed.key, Light, metadata)
            aio.send_data(intTempCfeed.key, InternalC, metadata)
            aio.send_data(intTempFfeed.key, InternalF, metadata)
            aio.send_data(extemp2Feed.key, External2, metadata)
            aio.send_data(extempFeed.key, External, metadata)
            aio.send_data(humidFeed.key, Humidity, metadata)
            aio.send_data(pressFeed.key, Pressure, metadata)
            
            print("Sending Data to the AdaFruit Cloud......")

    except IOError:
        pass
    
    except KeyboardInterrupt:
        print("Loop Terminated by User")
        
    except RuntimeError:
        print("Not Good")